In [ ]:
%env OPENAI_API_KEY=<PUT_YOUR_API_KEY_HERE>


In [4]:
import os
from openai import OpenAI
import faiss
from llama_index import SimpleDirectoryReader, ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.node_parser import SimpleNodeParser
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index import StorageContext, VectorStoreIndex

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

parser = SimpleNodeParser.from_defaults(separator="\n\n", chunk_size=100, chunk_overlap=20)
documents = SimpleDirectoryReader('./data/faq/').load_data()
nodes = parser.get_nodes_from_documents(documents)

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
))
service_context = ServiceContext.from_defaults(embed_model=embed_model)

dimension = 768
faiss_index = faiss.IndexFlatIP(dimension)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
storage_context.docstore.add_documents(nodes)
index = VectorStoreIndex(nodes=nodes, service_context=service_context, storage_context=storage_context)


In [5]:


query_index = index.as_query_engine(retriever_mode="embedding")

response = query_index.query(
    "What is FBA?",
)
print(response)


FBA stands for Fulfillment by Amazon (FBA). With FBA, you send products to Amazon’s fulfillment centers, and they pick, pack, ship, and provide customer service for those products.


In [6]:


query_index = index.as_query_engine(retriever_mode="embedding")

response = query_index.query(
    "What can I sell?",
)
print(response)


The possibilities are virtually limitless. What you can sell depends on the product, the product category, and the brand. Some categories are open to all sellers, some require a Professional seller account, some require approval to sell, and some include products that cannot be sold by third-party sellers.


In [7]:


query_index = index.as_query_engine(retriever_mode="embedding")

response = query_index.query(
    "What is Amazon Prime?",
)
print(response)


Amazon Prime is a program offered by Amazon that provides various benefits to its members. These benefits typically include free and fast shipping on eligible items, access to streaming services for movies, TV shows, and music, as well as exclusive deals and discounts. Additionally, Amazon Prime members may have access to other services such as Prime Video, Prime Music, Prime Reading, and more, depending on their location.


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig
tokenizer = AutoTokenizer.from_pretrained("../../../llm/Baichuan2-7B-Chat-4bits", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("../../../llm/Baichuan2-7B-Chat-4bits", device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)
model.generation_config = GenerationConfig.from_pretrained("../../../llm/Baichuan2-7B-Chat-4bits")


[2023-11-30 12:13:30,404] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-11-30 12:13:36.315630: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
messages = []
question = """
There are many opportunities for new sellers in Amazon’s store. What you can sell depends on the product, the category, and the brand. Some categories are open to all sellers while others require a Professional seller account. Certain products require approval to sell and other categories include products that cannot be sold by third-party sellers.
Based on the information above, please answer the following questions:
Q: What can I sell in Amazon’s store as a new seller?
"""
messages.append({"role": "user", "content": question})
response = model.chat(tokenizer, messages)
print(response)


As a new seller on Amazon, you can sell various products in different categories. However, certain products may require approval or be restricted to professional sellers. To determine what you can sell, it is important to research available categories, products, and brands.


In [12]:
messages = []
question = """
Q: What can I sell?
A:
"""
messages.append({"role": "user", "content": question})
response = model.chat(tokenizer, messages)
print(response)


You can sell various types of products or services, such as:

1. Consumer goods: clothing, electronics, home decor, toys, etc.
2. Services: cleaning, catering, transportation, entertainment, etc.
3. Digital products: eBooks, software, music, videos, etc.
4. Artwork and collectibles: paintings, sculptures, antiques, etc.
5. Real estate: selling or renting houses, apartments, land, etc.
6. Health and beauty products: skincare, hair care, supplements, etc.
7. Fitness and sports equipment: gym equipment, running shoes, etc.
8. Gifts and novelty items: key chains, mugs, t-shirts, etc.
9. Farm and garden products: fruits, vegetables, flowers, etc.
10. Local specialty products: regional foods, beverages, etc.

It's important to research your target market and find out what they are interested in buying. You can also create custom products based on your skills and interests.


In [13]:
messages = []
question = """
Top selling items on any given day may include books, baby items, electronics, kitchen supplies, outdoor gear—the list goes on. Check out Amazon Best Sellers for an hourly update with the most popular products based on sales.
Based on the information above, please answer the following questions:
Q: Are books a best-selling category of products?
"""
messages.append({"role": "user", "content": question})
response = model.chat(tokenizer, messages)
print(response)


Yes, books are a best-selling category of products.


In [14]:
messages = []
question = """
Top selling items on any given day may include books, baby items, electronics, kitchen supplies, outdoor gear—the list goes on. Check out Amazon Best Sellers for an hourly update with the most popular products based on sales.
Based on the information above, please answer the following questions:
Q: Are toys a best-selling category of products?
"""
messages.append({"role": "user", "content": question})
response = model.chat(tokenizer, messages)
print(response)


While toys are definitely a popular category of products, they are not always top sellers on any given day. This depends on various factors such as current trends, seasonal demand, and other factors affecting sales.
